In [4]:
%load_ext autoreload
%autoreload 2
%load_ext sql 
%pylab inline
%matplotlib inline

import matplotlib.text as text

import scipy.interpolate as interp
import pandas as pd
#from mp.txpepa import *
from sklearn import (manifold, datasets, decomposition, ensemble, lda,
                     random_projection)
from sklearn.metrics.pairwise import euclidean_distances,manhattan_distances
import statsmodels.api as sm
import numpy.linalg as LA
import bio.hts.traj as traj
from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import FloatVector
stats = importr('stats')
from sklearn.neighbors import KNeighborsClassifier

DAT_DIR = '/share/home/ishah/projects/Chem/data/tables/'
PKL_DIR = '/share/home/ishah/projects/Chem/data/pickle/'
RES_DIR = '/share/home/ishah/projects/Chem/data/results/'
FIG_DIR = '/share/home/ishah/projects/Chem/figs/readacross/'

import pickle

tmstmp = time.strftime("%m-%d-%Y",time.localtime())
tmstmp

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The sql extension is already loaded. To reload it, use:
  %reload_ext sql
Populating the interactive namespace from numpy and matplotlib


'08-28-2017'

In [8]:
from bio.data.genra import *

DB = openMongo(host='pb.epa.gov',user='ishah',passwd='ishah',db='genra_v3')
DB.collection_names()

[u'chm_fp',
 u'tox2_fp',
 u'tox_fp_0',
 u'tox_fp',
 u'bio_fp_info',
 u'genra_perf_v1',
 u'toxref',
 u'tox5_fp',
 u'chm_alerts',
 u'tx21_fp',
 u'compound',
 u'rattox_fp',
 u'bio_fp',
 u'fp_stats',
 u'tox4_fp',
 u'bio_fp_0',
 u'assay_info',
 u'genra_perf_v1_sum_v1',
 u'clusters1']

In [44]:
def calcPerf2(n_pos,n_neg,sens,spec,sn_sd=None,sp_sd=None):
    n = n_pos+n_neg
    TP = int(sens * n_pos)
    TN = int(spec * n_neg)
    FN = n_pos - TP
    FP = n_neg - TN
    
    if (TP+FP)==0 or (TN+FN)==0: return 
    
    PPV= 1.0*TP/(TP+FP)
    NPV= 1.0*TN/(TN+FN)
    FDR= 1 - PPV
    
    se=sp_se=sn_se=None
    
    if sn_sd and sp_sd:
        sn_se=sn_sd/np.sqrt(n)
        sp_se=sn_sd/np.sqrt(n)
        se = np.sqrt(sn_se**2 + sp_se**2)*np.sqrt(n)
        
    return dict(#n_pos=n_pos,n_neg=n_neg,
                TP=TP,TN=TN,FP=FP,FN=FN,
                #Sens=sens,Spec=spec,
                PPV = PPV, NPV=NPV, FDR=FDR,
                se=se
               )


In [14]:
q = {'auc':{'$gte':0.6},'n_pos':{'$gte':2},'n_neg':{'$gte':2}}
print DB.genra_perf_v1.find(q).count()
DB.genra_perf_v1.find_one(q)


175309


{u'_id': ObjectId('571a61a114fd094e29be93d4'),
 u'act_wt': 1,
 u'auc': 0.6,
 u'auc_pval': 0.0,
 u'ba_max': 0.6,
 u'cl': u'1',
 u'effect': u'DEV:Offspring Survival-Early',
 u'fpr_max': 0.0,
 u'k0': 1,
 u'n_cl': 47,
 u'n_neg': 3,
 u'n_pos': 5,
 u'n_sim_neg': 2,
 u'n_sim_pos': 5,
 u'n_tot': 8,
 u's0': 0.09999999999999998,
 u'sim_type': u'bc',
 u'sn_max': 0.2,
 u'sp_max': 1.0,
 u'study': u'DEV',
 u't0_max': 1.0}

In [23]:
P0 = DB.genra_perf_v1.find_one(q)
calcPerf2(P0['n_pos'],P0['n_neg'],P0['sn_max'],P0['sp_max']
         )

{'FDR': 0.0,
 'FN': 4,
 'FP': 0,
 'NPV': 0.42857142857142855,
 'PPV': 1.0,
 'TN': 3,
 'TP': 1,
 'se': None}

In [46]:
from util.jupyter import *

In [47]:
DB.genra_perf_v2.drop()
q = {'auc':{'$gte':0.2},'n_pos':{'$gte':2},'n_neg':{'$gte':2}}
N = DB.genra_perf_v1.find(q,dict(_id=0)).count()

for P0 in log_progress(DB.genra_perf_v1.find(q,dict(_id=0)),every=10,size=N):
    P1=calcPerf2(P0['n_pos'],P0['n_neg'],P0['sn_max'],P0['sp_max'])
    if not P1: continue
    P0.update(P1)
    DB.genra_perf_v2.insert_one(P0)

In [48]:
DB.genra_perf_v2.count()

237588

In [49]:
DB.genra_perf_v2.find_one()

{u'FDR': 0.0,
 u'FN': 4,
 u'FP': 0,
 u'NPV': 0.42857142857142855,
 u'PPV': 1.0,
 u'TN': 3,
 u'TP': 1,
 u'_id': ObjectId('59a42b91072e60a4e3e5623b'),
 u'act_wt': 1,
 u'auc': 0.2,
 u'auc_pval': 0.86,
 u'ba_max': 0.6,
 u'cl': u'1',
 u'effect': u'DEV:Uterus',
 u'fpr_max': 0.0,
 u'k0': 4,
 u'n_cl': 47,
 u'n_neg': 3,
 u'n_pos': 5,
 u'n_sim_neg': 3,
 u'n_sim_pos': 4,
 u'n_tot': 8,
 u's0': 0.09999999999999998,
 u'se': None,
 u'sim_type': u'chm',
 u'sn_max': 0.2,
 u'sp_max': 1.0,
 u'study': u'DEV',
 u't0_max': 1.0}

In [55]:
len(DB.genra_perf_v2.distinct('cl'))

36

In [76]:
r=dict(_id=0,se=0)
Perf0 = (pd.DataFrame(list(DB.genra_perf_v2.find({},r)))
         .rename(columns=dict(sim_type='dt',sn_max='SN_max',sp_max='SP_max',auc='AUC',PPV='PPV_max',NPV='NPV_max'))
        )

In [60]:
def Unq_Cl(x): return len(np.unique(x))

In [61]:
P1 = (Perf0.query("auc_pval<0.1")
      .groupby(['effect','dt'])
      .aggregate(dict(PPV=[np.median,np.std],
                      NPV=[np.median,np.std],
                      SN_max=[np.median,np.std],
                      SP_max=[np.median,np.std],
                      AUC=[np.median,np.std],
                      cl= Unq_Cl ))
     )
P1.head(30)

AUC                 PPV              SN_max  \
                         median       std    median       std    median   
effect            dt                                                      
CHR:Adrenal Gland bc   0.869141  0.144825  0.871429  0.198777  0.875000   
                  bct  0.831818  0.130435  0.881944  0.179580  0.909091   
                  bio  0.843750  0.130390  0.800000  0.175496  0.909091   
                  chm  0.872727  0.135168  0.909091  0.180982  0.848485   
                  cht  0.802189  0.130955  0.833333  0.178798  0.833333   
CHR:Blood         bc   0.961735  0.154359  1.000000  0.190628  0.928571   
                  bct  0.777778  0.140221  1.000000  0.212211  0.666667   
                  bio  0.773688  0.145500  1.000000  0.245670  0.571429   
                  chm  0.923469  0.141609  0.857143  0.222383  0.857143   
                  cht  0.700000  0.153737  0.857143  0.240431  0.666667   
CHR:Body Weight   bc   0.994147  0.133781  1.000000  0.029521  1.000000   
                  bct  0.912037  0.115213  1.000000  0.029518  1.000000   
                  bio  0.915970  0.110082  1.000000  0.025674  1.000000   
                  chm  0.953177  0.122887  0.989011  0.025839  0.994565   
                  cht  1.000000  0.115777  1.000000  0.025188  1.000000   
CHR:Bone          bc   1.000000  0.131685  1.000000  0.216759  1.000000   
                  bct  1.000000  0.144771  1.000000  0.349711  1.000000   
                  bio  0.913300  0.141664  1.000000  0.306775  0.833333   
                  chm  1.000000  0.106704  1.000000  0.209798  1.000000   
                  cht  1.000000  0.132943  1.000000  0.227048  1.000000   
CHR:Bone Marrow   bc   0.810741  0.136898  0.806250  0.229247  0.733333   
                  bct  0.742407  0.121087  0.666667  0.245688  0.666667   
                  bio  0.777778  0.132269  0.700893  0.227564  0.666667   
                  chm  0.911111  0.149058  1.000000  0.239416  0.866667   
                  cht  0.817460  0.138027  0.833333  0.251865  0.666667   
CHR:Brain         bc   0.906250  0.120817  0.968750  0.185230  1.000000   
                  bct  0.833333  0.141283  0.750000  0.204572  1.000000   
                  bio  0.837500  0.147623  0.823529  0.215526  0.935484   
                  chm  0.888405  0.132235  0.984375  0.188827  0.935484   
                  cht  0.875000  0.145198  0.800000  0.211245  1.000000   

                                    cl    SP_max                 NPV            
                            std Unq_Cl    median       std    median       std  
effect            dt                                                            
CHR:Adrenal Gland bc   0.174383     19  0.940972  0.166638  0.891892  0.123665  
                  bct  0.190085     20  0.900000  0.163766  0.943396  0.124748  
                  bio  0.184796     22  0.833333  0.154527  0.933333  0.141263  
                  chm  0.168488     20  0.958333  0.147973  0.900000  0.138479  
                  cht  0.173811     21  0.888889  0.165446  0.882353  0.147734  
CHR:Blood         bc   0.253750      5  1.000000  0.087408  0.994898  0.111091  
                  bct  0.250915      4  1.000000  0.048429  0.888889  0.094098  
                  bio  0.254383      4  1.000000  0.041742  0.970297  0.058794  
                  chm  0.204004      5  0.989796  0.125111  0.989796  0.107659  
                  cht  0.228966      5  0.989796  0.120442  0.960000  0.113903  
CHR:Body Weight   bc   0.021922      3  1.000000  0.238987  1.000000  0.139223  
                  bct  0.112873      3  1.000000  0.192058  1.000000  0.279416  
                  bio  0.109669      3  1.000000  0.183937  1.000000  0.271519  
                  chm  0.043738      3  0.923077  0.199083  0.961538  0.228625  
                  cht  0.031677      3  1.000000  0.201267  1.000000  0.177170  
CHR:Bone          bc   0.219833      3  1.000000  0.030597  1.000000  0.054066  
             

In [91]:
P1 = (Perf0#.query("auc_pval<0.1")
      .groupby(['effect','dt'])
      .aggregate(dict(PPV_max=dict(mn=np.mean,sd=np.std),
                      NPV_max=dict(mn=np.mean,sd=np.std),
                      SN_max=dict(mn=np.mean,sd=np.std),
                      SP_max=dict(mn=np.mean,sd=np.std),
                      AUC=dict(mn=np.mean,sd=np.std),
                      cl= Unq_Cl))
     )
P1.head(30)

AUC             NPV_max              SN_max  \
                             mn        sd        mn        sd        mn   
effect            dt                                                      
CHR:Adrenal Gland bc   0.694372  0.210197  0.813547  0.174539  0.744399   
                  bct  0.685478  0.222679  0.832411  0.171150  0.765761   
                  bio  0.677514  0.213550  0.805244  0.178989  0.726480   
                  chm  0.723943  0.210841  0.824985  0.161345  0.777135   
                  cht  0.705329  0.181371  0.822507  0.157196  0.748605   
CHR:Blood         bc   0.695449  0.239853  0.905874  0.144547  0.708738   
                  bct  0.713452  0.137518  0.904644  0.092709  0.626116   
                  bio  0.714385  0.190337  0.915210  0.098311  0.597418   
                  chm  0.725462  0.206623  0.904682  0.129131  0.715631   
                  cht  0.714787  0.169430  0.899066  0.119112  0.640751   
CHR:Body Weight   bc   0.780254  0.210075  0.722438  0.351914  0.911405   
                  bct  0.788409  0.216618  0.659790  0.358345  0.876846   
                  bio  0.856533  0.161870  0.735780  0.316072  0.906355   
                  chm  0.814717  0.198445  0.710612  0.336338  0.912026   
                  cht  0.854194  0.172832  0.813751  0.294534  0.966957   
CHR:Bone          bc   0.875947  0.165906  0.972804  0.051175  0.793561   
                  bct  0.819340  0.219534  0.988653  0.016603  0.826797   
                  bio  0.770252  0.192351  0.966358  0.050245  0.715608   
                  chm  0.904676  0.158056  0.989924  0.016229  0.840136   
                  cht  0.858414  0.173782  0.983455  0.019891  0.759259   
CHR:Bone Marrow   bc   0.692352  0.199088  0.892515  0.102781  0.677300   
                  bct  0.692124  0.152560  0.880288  0.103090  0.683838   
                  bio  0.690573  0.166366  0.899619  0.109209  0.712736   
                  chm  0.763963  0.203049  0.912663  0.105681  0.730392   
                  cht  0.732669  0.175341  0.890022  0.100364  0.679419   
CHR:Brain         bc   0.741522  0.225562  0.929706  0.096471  0.834591   
                  bct  0.706257  0.214851  0.914021  0.123190  0.819371   
                  bio  0.698964  0.211954  0.896395  0.134456  0.782800   
                  chm  0.761635  0.212272  0.901302  0.124577  0.806017   
                  cht  0.718540  0.221498  0.904564  0.121182  0.799682   

                                    cl    SP_max             PPV_max            
                             sd Unq_Cl        mn        sd        mn        sd  
effect            dt                                                            
CHR:Adrenal Gland bc   0.242701     21  0.778130  0.222104  0.753099  0.231375  
                  bct  0.249861     21  0.770994  0.242245  0.775432  0.220759  
                  bio  0.264596     22  0.772311  0.222168  0.757162  0.212887  
                  chm  0.210139     21  0.787353  0.224494  0.773491  0.211028  
                  cht  0.224065     22  0.776456  0.208573  0.767747  0.210084  
CHR:Blood         bc   0.280298      5  0.856172  0.206516  0.733474  0.305290  
                  bct  0.250425      5  0.944962  0.118765  0.880208  0.191229  
                  bio  0.253486      4  0.958660  0.125740  0.882294  0.266091  
                  chm  0.251012      5  0.867621  0.144250  0.706744  0.286052  
                  cht  0.262539      5  0.893554  0.137638  0.728348  0.285569  
CHR:Body Weight   bc   0.143481      3  0.709057  0.279142  0.954988  0.045279  
                  bct  0.177992      3  0.768498  0.267859  0.960972  0.043632  
                  bio  0.162682      3  0.841624  0.226227  0.976939  0.032699  
                  chm  0.138131      3  0.764927  0.223481  0.966521  0.033435  
                  cht  0.060892      3  0.760225  0.275392  0.967477  0.038599  
CHR:Bone          bc   0.265026      3  0.981543  0.040912  0.847658  0.315225  
             

In [93]:
Res = []
for effect,Pi in P1.groupby(level=0):
    Res.append(Pi.sort_values(('AUC','mn')).tail(1))
    print effect,dt,Pi.shape

CHR:Adrenal Gland cht (5, 11)
CHR:Blood cht (5, 11)
CHR:Body Weight cht (5, 11)
CHR:Bone cht (5, 11)
CHR:Bone Marrow cht (5, 11)
CHR:Brain cht (5, 11)
CHR:Clinical Chemistry cht (5, 11)
CHR:Clinical Signs cht (5, 11)
CHR:Ear cht (5, 11)
CHR:Epididymis cht (5, 11)
CHR:Eye cht (5, 11)
CHR:Food Consumption cht (5, 11)
CHR:Gallbladder cht (5, 11)
CHR:Harderian Gland cht (5, 11)
CHR:Heart cht (5, 11)
CHR:Hematology cht (5, 11)
CHR:Intestine Large cht (5, 11)
CHR:Intestine Small cht (5, 11)
CHR:Kidney cht (5, 11)
CHR:Lacrimal Gland cht (5, 11)
CHR:Liver cht (5, 11)
CHR:Lung cht (5, 11)
CHR:Lymph Node cht (5, 11)
CHR:Mammary Gland cht (5, 11)
CHR:Mortality cht (5, 11)
CHR:Nerve cht (5, 11)
CHR:Nose cht (5, 11)
CHR:Ovary cht (5, 11)
CHR:Pancreas cht (5, 11)
CHR:Parathyroid cht (5, 11)
CHR:Parathyroid Gland cht (5, 11)
CHR:Penis cht (5, 11)
CHR:Pituitary Gland cht (5, 11)
CHR:Preputial Gland cht (5, 11)
CHR:Prostate cht (5, 11)
CHR:Salivary glands cht (5, 11)
CHR:Seminal Vesicle cht (5, 11)
CHR

In [103]:
P2 = pd.concat(Res)

In [104]:
P2 = (P2[['cl','AUC','SN_max','PPV_max','SP_max']]
      .round(decimals=2)
      .rename(dict(SN_max='sens',SP_max='spec',PPV_max='ppv'))
     )

In [105]:
P2.head()

cl   AUC       SN_max       PPV_max       SP_max  \
                      Unq_Cl    mn    sd     mn    sd      mn    sd     mn   
effect            dt                                                         
CHR:Adrenal Gland chm     21  0.72  0.21   0.78  0.21    0.77  0.21   0.79   
CHR:Blood         chm      5  0.73  0.21   0.72  0.25    0.71  0.29   0.87   
CHR:Body Weight   bio      3  0.86  0.16   0.91  0.16    0.98  0.03   0.84   
CHR:Bone          chm      2  0.90  0.16   0.84  0.26    0.86  0.31   0.98   
CHR:Bone Marrow   chm     12  0.76  0.20   0.73  0.27    0.74  0.29   0.88   

                             
                         sd  
effect            dt         
CHR:Adrenal Gland chm  0.22  
CHR:Blood         chm  0.14  
CHR:Body Weight   bio  0.23  
CHR:Bone          chm  0.04  
CHR:Bone Marrow   chm  0.16

In [109]:
C = [i+'_'+j for (i,j) in list(P2.columns)]
P2.columns=C


In [110]:
P2.head()

,,cl_Unq_Cl,AUC_mn,AUC_sd,SN_max_mn,SN_max_sd,PPV_max_mn,PPV_max_sd,SP_max_mn,SP_max_sd
effect,dt,,,,,,,,,
CHR:Adrenal Gland,chm,21,0.72,0.21,0.78,0.21,0.77,0.21,0.79,0.22
CHR:Blood,chm,5,0.73,0.21,0.72,0.25,0.71,0.29,0.87,0.14
CHR:Body Weight,bio,3,0.86,0.16,0.91,0.16,0.98,0.03,0.84,0.23
CHR:Bone,chm,2,0.90,0.16,0.84,0.26,0.86,0.31,0.98,0.04
CHR:Bone Marrow,chm,12,0.76,0.20,0.73,0.27,0.74,0.29,0.88,0.16


In [113]:
P2['res2_sn']=P2.apply(lambda x: "%3.2f±%3.2f" % (x.SN_max_mn,x.SN_max_sd),axis=1)
P2['res2_sp']=P2.apply(lambda x: "%3.2f±%3.2f" % (x.SP_max_mn,x.SP_max_sd),axis=1)
P2['res2_ppv']=P2.apply(lambda x: "%3.2f±%3.2f" % (x.PPV_max_mn,x.PPV_max_sd),axis=1)
P2['res2_auc']=P2.apply(lambda x: "%3.2f±%3.2f" % (x.AUC_mn,x.AUC_sd),axis=1)

In [114]:
P2.head()

,,cl_Unq_Cl,AUC_mn,AUC_sd,SN_max_mn,SN_max_sd,PPV_max_mn,PPV_max_sd,SP_max_mn,SP_max_sd,res2_sn,res2_sp,res2_ppv,res2_auc
effect,dt,,,,,,,,,,,,,
CHR:Adrenal Gland,chm,21,0.72,0.21,0.78,0.21,0.77,0.21,0.79,0.22,0.78±0.21,0.79±0.22,0.77±0.21,0.72±0.21
CHR:Blood,chm,5,0.73,0.21,0.72,0.25,0.71,0.29,0.87,0.14,0.72±0.25,0.87±0.14,0.71±0.29,0.73±0.21
CHR:Body Weight,bio,3,0.86,0.16,0.91,0.16,0.98,0.03,0.84,0.23,0.91±0.16,0.84±0.23,0.98±0.03,0.86±0.16
CHR:Bone,chm,2,0.90,0.16,0.84,0.26,0.86,0.31,0.98,0.04,0.84±0.26,0.98±0.04,0.86±0.31,0.90±0.16
CHR:Bone Marrow,chm,12,0.76,0.20,0.73,0.27,0.74,0.29,0.88,0.16,0.73±0.27,0.88±0.16,0.74±0.29,0.76±0.20


In [126]:
X = P2.sort_values('AUC_mn',ascending=False)[['cl_Unq_Cl','res2_auc','res2_sn','res2_sp','res2_ppv']]

#X.columns = "Classifier ID	Study:Target	dt	n	ML Alg	F1	Sensitivity	Specificity	PPV".split("\t")

X.to_csv('/share/home/ishah/tmp/genra-perf-with-ppv-summary-v2.csv')

#ew = pd.ExcelWriter('/share/home/ishah/tmp/genra-perf-with-ppv-summary-v2.xlsx', )
#X.to_excel(ew)
#ew.save()
#for x in [i for i in X if i.startswith('res2')]:
#    X[x] = X[x].str.decode('ascii').str.encode('utf-8')
    

